# MSA 2025 Phase 2 - Part 1

In [1]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Find all variables and understand them

In [ ]:
#
print("1")
print("2")


1
2


## 2. Clean data

In [3]:
#

## 3. Visualise data

In [4]:
#

## 4. Identify correlated variables

In [5]:
#

## 5. Summary